In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from os.path import join as pjoin
from natsort import natsorted
import statsmodels.api as sm
from statsmodels.formula.api import ols
from numpy.random import MT19937, SeedSequence, RandomState

sys.path.append("../../")
import circletrack_behavior as ctb
import plotting_functions as pf

In [ ]:
## Settings
parent_dir = 'MultiCon_Behavior'
experiment_dir = 'MultiCon_Lesion4'
exp_path = f'../../../{parent_dir}'
lin_path = f'../../../{parent_dir}/{experiment_dir}/output/lin_behav/'
circle_path = f'../../../{parent_dir}/{experiment_dir}/output/behav/'
fig_path = f'../../../{parent_dir}/{experiment_dir}/intermediate_figures'
maze_info = pd.read_csv(f'../../../{parent_dir}/{experiment_dir}/maze_yml/maze_info.csv')
chance_color = 'darkgrey'
avg_color = 'midnightblue'
subject_color = 'darkgrey'
two_group_colors = ['red', 'blue']
three_group_colors = ['red', 'blue', 'darkgrey']
four_group_colors = ['darkorchid', 'midnightblue', 'orchid', 'blue']
group_colors_dict = {'Sham': 'blue', 'HPC': 'red', 'mPFC': 'darkgrey'}
error_dict = {'Sham': 'rgba(0,41,102,0.4)', 'HPC': 'rgba(153,50,204,0.4)',
              'mPFC': 'rgba(169,169,169,0.4)'}
symbol_list = ['circle', 'circle', 'circle']
excluded_mice = ['mcls70'] ## excluded for arduino not registering any of this mouse's licks on the recall day

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

rs = RandomState(MT19937(SeedSequence(24601)))

### Circle track lick accuracy and rewards across days.

In [ ]:
circletrack_results = {'mouse': [], 'day': [], 'sex': [], 'group': [], 'session': [], 'lick_accuracy': [], 'rewards': []}
for mouse in os.listdir(circle_path):
    mouse_path = pjoin(circle_path, mouse)
    sex = maze_info['Sex'][maze_info['Mouse'] == mouse].values[0]
    group = maze_info['Group'][maze_info['Mouse'] == mouse].values[0]
    if 'Sham' in group:
        group = 'Sham'
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        behav = behav[~behav['probe']]
        reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
        pc_thresh5 = ctb.lick_accuracy(behav, port_list=[reward_one, reward_two], lick_threshold=5, by_trials=False)
        circletrack_results['mouse'].append(mouse)
        circletrack_results['day'].append(idx+1)
        circletrack_results['sex'].append(sex)
        circletrack_results['group'].append(group)
        circletrack_results['session'].append(np.unique(behav['session'])[0])
        circletrack_results['lick_accuracy'].append(pc_thresh5)
        circletrack_results['rewards'].append(np.sum(behav['water']))
ct_df = pd.DataFrame(circletrack_results)

In [ ]:
## Plot 5th lick accuracy across days
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='lick_accuracy', groupby_var=['day'], plot_transitions=None,
                                   marker_color=avg_color, avg_color=avg_color, expert_line=False, chance=True, symbols=['circle'],
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 100])
fig.show()

In [ ]:
## Plot rewards across days
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='rewards', groupby_var=['day'], plot_transitions=None,
                                   marker_color=avg_color, avg_color=avg_color, expert_line=False, chance=False, symbols=['circle'],
                                   x_title='Day', y_title='Rewards', titles=['Circle Track'], height=500, width=500)
fig.show()

In [ ]:
## Plot 5th lick accuracy across days for the three groups
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='lick_accuracy', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_group_colors, avg_color=avg_color, expert_line=False, chance=True, symbols=symbol_list,
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 100])
fig.show()

In [ ]:
## Plot rewards across days for the three groups
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='rewards', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_group_colors, avg_color=avg_color, expert_line=False, chance=False, symbols=symbol_list,
                                   x_title='Day', y_title='Rewards', titles=['Circle Track'], height=500, width=500)
fig.show()

In [ ]:
## Plot 5th lick accuracy across days for the Sham and HPC groups
sub_df = ct_df[ct_df['group'] != 'mPFC'].reset_index(drop=True)
fig = pf.plot_behavior_across_days(sub_df[sub_df['day'] < 6], x_var='day', y_var='lick_accuracy', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=two_group_colors, avg_color=avg_color, expert_line=False, chance=True, symbols=symbol_list,
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=['Training'], 
                                   plot_datapoints=False, height=500, width=500)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'lick_acc_hpc_sham.png'), width=500, height=500)

### Look at probe accuracy.

In [ ]:
lick_dict_probe = {'mouse': [], 'experiment': [], 'sex': [], 'group': [], 'session': [], 
                   'day': [], 'num_licks': [], 'probe_acc': [], 'session_acc': []}
for mouse in os.listdir(circle_path):
    mpath = pjoin(circle_path, mouse)
    sex = maze_info['Sex'][maze_info['Mouse'] == mouse].values[0]
    group = maze_info['Group'][maze_info['Mouse'] == mouse].values[0]
    if 'Sham' in group:
        group = 'Sham'
    for idx, session in enumerate(os.listdir(mpath)):
        behav = pd.read_feather(pjoin(mpath, session))
        if any(behav['probe']):
            behav_probe = behav[behav['probe']]
            behav_no_probe = behav[~behav['probe']]
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            percent_correct = ctb.lick_accuracy(behav_probe, port_list=[reward_one, reward_two], lick_threshold=5, by_trials=False)
            session_pc = ctb.lick_accuracy(behav_no_probe, port_list=[reward_one, reward_two], lick_threshold=5, by_trials=False)
            lick_dict_probe['mouse'].append(mouse)
            lick_dict_probe['experiment'].append(behav['cohort'].unique()[0])
            lick_dict_probe['sex'].append(sex)
            lick_dict_probe['group'].append(group)
            lick_dict_probe['day'].append(idx+1)
            lick_dict_probe['session'].append(np.unique(behav['session'])[0])
            lick_dict_probe['num_licks'].append(len(behav_probe[behav_probe['lick_port'] != -1]))
            lick_dict_probe['probe_acc'].append(percent_correct)
            lick_dict_probe['session_acc'].append(session_pc)
        else:
            pass
probe_df = pd.DataFrame(lick_dict_probe)

In [ ]:
## Plot last day probe accuracy for each group
probe_df = probe_df.replace({'AR': 'Recall'})
avg_last = probe_df.groupby(['session', 'group'], as_index=False).agg({'probe_acc': ['mean', 'sem']})
fig = pf.custom_graph_template(x_title='', y_title='Lick Accuracy (%)', titles=['Probe Accuracy'], width=500)
for group in avg_last['group'].unique():
    gdata = avg_last[avg_last['group'] == group].reset_index(drop=True)
    fig.add_trace(go.Scatter(x=gdata['session'], y=gdata['probe_acc']['mean'], showlegend=False, name=group,
                             error_y=dict(type='data', array=gdata['probe_acc']['sem']), legendgroup=group,
                             mode='markers', marker_color=group_colors_dict[group]))
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color='darkgrey', opacity=1)
fig.update_yaxes(range=[0, 100])
for val in [0, 1, 2]:
    fig.data[val]['showlegend'] = True
fig.show()

### Look at accuracy across trials.

In [ ]:
bin_size = 4
lick_thresh = 5
trial_res = {'mouse': [], 'sex': [], 'group': [], 'day': [], 'session': [], 'session_two': [], 'trial': [], 'lick_acc': []}
for mouse in os.listdir(circle_path):
    mpath = pjoin(circle_path, mouse)
    sex = maze_info['Sex'][maze_info['Mouse'] == mouse].values[0]
    group = maze_info['Group'][maze_info['Mouse'] == mouse].values[0]
    if 'Sham' in group:
        group = 'Sham'
    for idx, session in enumerate(os.listdir(mpath)):
        behav = pd.read_feather(pjoin(mpath, f'{session}'))
        reward_one, reward_two = behav['reward_one'].unique()[0], behav['reward_two'].unique()[0]
        trial_acc = ctb.lick_accuracy(behav, port_list=[reward_one, reward_two], lick_threshold=lick_thresh, by_trials=True)
        
        if bin_size > 0:
            binned_acc = ctb.bin_data(trial_acc, bin_size)
        else:
            binned_acc = trial_acc

        for trial, val in enumerate(binned_acc):
            trial_res['mouse'].append(mouse)
            trial_res['sex'].append(sex)
            trial_res['group'].append(group)
            trial_res['day'].append(idx+1)
            trial_res['session'].append(behav['session'].unique()[0])
            trial_res['session_two'].append(behav['session_two'].unique()[0])
            trial_res['trial'].append(trial+1)
            trial_res['lick_acc'].append(val)
trial_df = pd.DataFrame(trial_res)

In [ ]:
## Plot accuracy across trials for all days
trial_df = trial_df.replace({'AR1': 'Recall'})
day_list = ['A1', 'A2', 'A3', 'A4', 'A5', 'Recall']
only_min_trials = False
ar = trial_df.copy()
avg_ar = ar.groupby(['group', 'session_two', 'trial'], as_index=False).agg({'lick_acc': ['mean', 'sem']})

fig = pf.custom_graph_template(x_title='Trials', y_title='', rows=1, columns=6, shared_x=True, width=1200, height=500,
                               shared_y=True, titles=day_list)
for group in ['Sham', 'HPC', 'mPFC']:
    gdata = avg_ar[avg_ar['group'] == group].reset_index(drop=True)  
    for idx, sess in enumerate(day_list):
        trial_bins = []
        if only_min_trials:
            int_data = ar[(ar['group'] == group) & (ar['session_two'] == sess)]
            for mouse in int_data['mouse'].unique():
                d = int_data[int_data['mouse'] == mouse].reset_index(drop=True)
                trial_bins.append(d['trial'].shape[0])
                min_trials = np.min(trial_bins)
        else:
            min_trials = None

        if min_trials is not None:
            sess_data = gdata[gdata['session_two'] == sess].reset_index(drop=True)[:min_trials]
            xaxis = np.arange(1, sess_data['trial'].to_numpy()[-1] * bin_size, bin_size)[:min_trials]
            upper = sess_data['lick_acc']['mean'] + sess_data['lick_acc']['sem']
            lower = sess_data['lick_acc']['mean'] - sess_data['lick_acc']['sem']
        else:
            sess_data = gdata[gdata['session_two'] == sess].reset_index(drop=True)
            xaxis = np.arange(1, sess_data['trial'].to_numpy()[-1] * bin_size, bin_size)
            upper = sess_data['lick_acc']['mean'] + sess_data['lick_acc']['sem']
            lower = sess_data['lick_acc']['mean'] - sess_data['lick_acc']['sem']
        fig.add_trace(go.Scatter(x=xaxis, y=sess_data['lick_acc']['mean'], mode='lines', line_color=group_colors_dict[group],
                                   name=group, showlegend=False, legendgroup=group), row=1, col=idx+1)
        fig.add_trace(go.Scatter(x=xaxis, y=upper, mode='lines', marker=dict(color=error_dict[group]), legendgroup=group,
                                        name='Upper Bound', line=dict(width=0), showlegend=False), row=1, col=idx+1)
        fig.add_trace(go.Scatter(x=xaxis, y=lower, mode='lines', marker=dict(color=error_dict[group]), legendgroup=group,
                                name='Lower Bound', line=dict(width=0), showlegend=False, fillcolor=error_dict[group], fill='tonexty'), row=1, col=idx+1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color='darkgrey', opacity=1)
fig.update_yaxes(range=[0, 100])
fig.update_yaxes(title='Lick Accuracy (%)', col=1)
for val in [0, 18, 36]:
    fig.data[val]['showlegend'] = True
fig.show()

In [ ]:
## Plot accuracy across trials for just recall
trial_df = trial_df.replace({'AR1': 'Recall'})
day_list = ['Recall']
only_min_trials = False
ar = trial_df.copy()
avg_ar = ar.groupby(['group', 'session_two', 'trial'], as_index=False).agg({'lick_acc': ['mean', 'sem']})

fig = pf.custom_graph_template(x_title='Trials', y_title='', rows=1, columns=1, shared_x=True, width=500, height=500,
                               shared_y=True, titles=day_list)
for group in ['Sham', 'HPC', 'mPFC']:
    gdata = avg_ar[avg_ar['group'] == group].reset_index(drop=True)  
    for idx, sess in enumerate(day_list):
        trial_bins = []
        if only_min_trials:
            int_data = ar[(ar['group'] == group) & (ar['session_two'] == sess)]
            for mouse in int_data['mouse'].unique():
                d = int_data[int_data['mouse'] == mouse].reset_index(drop=True)
                trial_bins.append(d['trial'].shape[0])
                min_trials = np.min(trial_bins)
        else:
            min_trials = None

        if min_trials is not None:
            sess_data = gdata[gdata['session_two'] == sess].reset_index(drop=True)[:min_trials]
            xaxis = np.arange(1, sess_data['trial'].to_numpy()[-1] * bin_size, bin_size)[:min_trials]
            upper = sess_data['lick_acc']['mean'] + sess_data['lick_acc']['sem']
            lower = sess_data['lick_acc']['mean'] - sess_data['lick_acc']['sem']
        else:
            sess_data = gdata[gdata['session_two'] == sess].reset_index(drop=True)
            xaxis = np.arange(1, sess_data['trial'].to_numpy()[-1] * bin_size, bin_size)
            upper = sess_data['lick_acc']['mean'] + sess_data['lick_acc']['sem']
            lower = sess_data['lick_acc']['mean'] - sess_data['lick_acc']['sem']
        fig.add_trace(go.Scatter(x=xaxis, y=sess_data['lick_acc']['mean'], mode='lines', line_color=group_colors_dict[group],
                                   name=group, showlegend=False, legendgroup=group), row=1, col=idx+1)
        fig.add_trace(go.Scatter(x=xaxis, y=upper, mode='lines', marker=dict(color=error_dict[group]), legendgroup=group,
                                        name='Upper Bound', line=dict(width=0), showlegend=False), row=1, col=idx+1)
        fig.add_trace(go.Scatter(x=xaxis, y=lower, mode='lines', marker=dict(color=error_dict[group]), legendgroup=group,
                                name='Lower Bound', line=dict(width=0), showlegend=False, fillcolor=error_dict[group], fill='tonexty'), row=1, col=idx+1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color='darkgrey', opacity=1)
fig.update_yaxes(range=[0, 100])
fig.update_yaxes(title='Lick Accuracy (%)', col=1)
for val in [0, 3, 6]:
    fig.data[val]['showlegend'] = True
fig.show()

### Combine Lesion3 and Lesion4 and plot the same graphs

In [ ]:
circletrack_results = {'mouse': [], 'day': [], 'sex': [], 'group': [], 'session': [], 'lick_accuracy': [], 'rewards': []}
for exp in ['MultiCon_Lesion3', 'MultiCon_Lesion4']:
    dpath = pjoin(exp_path, f'{exp}/output/behav')
    maze_info = pd.read_csv(f'../../../{parent_dir}/{exp}/maze_yml/maze_info.csv')
    for mouse in os.listdir(dpath):
        if mouse in excluded_mice:
            pass
        else:
            mpath = pjoin(dpath, mouse)
            sex = maze_info['Sex'][maze_info['Mouse'] == mouse].values[0]
            group = maze_info['Group'][maze_info['Mouse'] == mouse].values[0]
            if 'Sham' in group:
                group = 'Sham'
            for idx, session in enumerate(os.listdir(mpath)):
                behav = pd.read_feather(pjoin(mpath, f'{session}'))
                behav = behav[~behav['probe']]
                reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                pc_thresh5 = ctb.lick_accuracy(behav, port_list=[reward_one, reward_two], lick_threshold=5, by_trials=False)
                circletrack_results['mouse'].append(mouse)
                circletrack_results['day'].append(idx+1)
                circletrack_results['sex'].append(sex)
                circletrack_results['group'].append(group)
                circletrack_results['session'].append(np.unique(behav['session'])[0])
                circletrack_results['lick_accuracy'].append(pc_thresh5)
                circletrack_results['rewards'].append(np.sum(behav['water']))
ct_df = pd.DataFrame(circletrack_results)

In [ ]:
## Plot 5th lick accuracy across days for the three groups
fig = pf.plot_behavior_across_days(ct_df[ct_df['day'] < 7], x_var='day', y_var='lick_accuracy', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_group_colors, avg_color=avg_color, expert_line=False, chance=True, symbols=symbol_list,
                                   plot_datapoints=False,
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 100])
fig.show()

In [ ]:
## Plot 5th lick accuracy for just HPC and Sham mice
fig = pf.plot_behavior_across_days(ct_df[(ct_df['day'] < 6) & (ct_df['group'] != 'mPFC')], x_var='day', y_var='lick_accuracy', groupby_var=['day', 'group'], 
                                   plot_transitions=None, plot_datapoints=False,
                                   marker_color=three_group_colors, avg_color=avg_color, expert_line=False, chance=True, symbols=symbol_list,
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=[''], height=500, width=500)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'lick_acc_HPC_Sham.png'), width=500, height=500)

In [ ]:
## Plot rewards across days for the three groups
fig = pf.plot_behavior_across_days(ct_df[ct_df['day'] < 7], x_var='day', y_var='rewards', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_group_colors, avg_color=avg_color, expert_line=False, chance=False, symbols=symbol_list,
                                   plot_datapoints=False,
                                   x_title='Day', y_title='Rewards', titles=['Circle Track'], height=500, width=500)
fig.show()

In [ ]:
## Plot rewards across days for HPC and Sham
fig = pf.plot_behavior_across_days(ct_df[(ct_df['day'] < 6) & (ct_df['group'] != 'mPFC')], x_var='day', y_var='rewards', groupby_var=['day', 'group'], 
                                   plot_transitions=None, plot_datapoints=False,
                                   marker_color=three_group_colors, avg_color=avg_color, expert_line=False, chance=False, symbols=symbol_list,
                                   x_title='Day', y_title='Rewards', titles=['Circle Track'], height=500, width=500)
fig.show()
fig.write_image(pjoin(fig_path, 'rewards_HPC_Sham.png'))

In [ ]:
lick_dict_probe = {'mouse': [], 'experiment': [], 'sex': [], 'group': [], 'session': [], 
                   'day': [], 'num_licks': [], 'probe_acc': [], 'session_acc': []}
for exp in ['MultiCon_Lesion3', 'MultiCon_Lesion4']:
    dpath = pjoin(exp_path, f'{exp}/output/behav')
    maze_info = pd.read_csv(f'../../../{parent_dir}/{exp}/maze_yml/maze_info.csv')
    for mouse in os.listdir(dpath):
        if mouse in excluded_mice:
            pass 
        else:
            mpath = pjoin(dpath, mouse)
            sex = maze_info['Sex'][maze_info['Mouse'] == mouse].values[0]
            group = maze_info['Group'][maze_info['Mouse'] == mouse].values[0]
            if 'Sham' in group:
                group = 'Sham'
            for idx, session in enumerate(os.listdir(mpath)):
                behav = pd.read_feather(pjoin(mpath, session))
                if any(behav['probe']):
                    behav_probe = behav[behav['probe']]
                    behav_no_probe = behav[~behav['probe']]
                    reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                    percent_correct = ctb.lick_accuracy(behav_probe, port_list=[reward_one, reward_two], lick_threshold=5, by_trials=False)
                    if np.isnan(percent_correct):
                        percent_correct = ctb.lick_accuracy(behav_probe, port_list=[reward_one, reward_two], lick_threshold=1, by_trials=False)
                    session_pc = ctb.lick_accuracy(behav_no_probe, port_list=[reward_one, reward_two], lick_threshold=5, by_trials=False)
                    lick_dict_probe['mouse'].append(mouse)
                    lick_dict_probe['experiment'].append(behav['cohort'].unique()[0])
                    lick_dict_probe['sex'].append(sex)
                    lick_dict_probe['group'].append(group)
                    lick_dict_probe['day'].append(idx+1)
                    lick_dict_probe['session'].append(np.unique(behav['session'])[0])
                    lick_dict_probe['num_licks'].append(len(behav_probe[behav_probe['lick_port'] != -1]))
                    lick_dict_probe['probe_acc'].append(percent_correct)
                    lick_dict_probe['session_acc'].append(session_pc)
                else:
                    pass
probe_df_combined = pd.DataFrame(lick_dict_probe)

In [ ]:
## Plot recall probe accuracy for each group
probe_df_combined = probe_df_combined.replace({'AR': 'Recall'})
probe_df_combined = probe_df_combined[probe_df_combined['session'] != 'B'] ## lesion4 was probed on first day in B
avg_last = probe_df_combined.groupby(['session', 'group'], as_index=False).agg({'probe_acc': ['mean', 'sem']})
fig = pf.custom_graph_template(x_title='', y_title='Lick Accuracy (%)', titles=['Probe Performance'], width=500)
for group in ['Sham', 'HPC']:
    gdata = avg_last[avg_last['group'] == group].reset_index(drop=True)
    fig.add_trace(go.Scatter(x=gdata['session'], y=gdata['probe_acc']['mean'], showlegend=False, name=group,
                             error_y=dict(type='data', array=gdata['probe_acc']['sem']), legendgroup=group,
                             mode='markers', marker_color=group_colors_dict[group]))
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(range=[0, 100])
for val in [0, 1]:
    fig.data[val]['showlegend'] = True
fig.show()
fig.write_image(pjoin(fig_path, 'probe_accuracy_2group.png'))

In [ ]:
## Plot recall probe accuracy for each group as a boxplot
probe_df_combined = probe_df_combined.replace({'AR': 'Recall'})
probe_df_combined = probe_df_combined[probe_df_combined['session'] != 'B'] ## lesion4 was probed on first day in B
fig = pf.custom_graph_template(x_title='', y_title='Lick Accuracy (%)', titles=['Non-Rewarded Probe Accuracy'])
for group in ['Sham', 'HPC']:
    gdata = probe_df_combined[probe_df_combined['group'] == group]
    fig.add_trace(go.Box(y=gdata['probe_acc'], boxpoints='all', jitter=0.8, pointpos=-2, opacity=0.7,
                         marker_color=group_colors_dict[group], name=group, showlegend=False))
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_traces(width=0.3)
fig.show()
fig.write_image(pjoin(fig_path, 'probe_performance_boxplots_2group.png'), width=500, height=500)

In [ ]:
## Plot recall probe accuracy for sham and HPC as a barplot
## Have to do some crazy shit in Plotly to plot two groups with jitter
## Run 4 times to get plot with correct jitter
probe_df_combined = probe_df_combined.replace({'AR': 'Recall'})
probe_df_combined = probe_df_combined[(probe_df_combined['session'] != 'B') & (probe_df_combined['group'] != 'mPFC')] ## lesion4 was probed on first day in B
avg_probe = probe_df_combined.groupby(['group', 'day'], as_index=False).agg({'probe_acc': ['mean', 'sem']})
avg_probe.loc[avg_probe['group'] == 'Sham', 'day'] = 5
shifts = rs.uniform(-0.12, 0.12, probe_df_combined['mouse'].unique().shape[0])

fig = pf.custom_graph_template(x_title='', y_title='Lick Accuracy (%)', titles=[''])
fig.add_trace(go.Bar(x=avg_probe['day'], y=avg_probe['probe_acc']['mean'], marker_color=two_group_colors,
                     marker_line_color='black', marker_line_width=2, opacity=0.8, showlegend=False,
                     error_y=dict(type='data', array=avg_probe['probe_acc']['sem']), width=0.4))
for idx, mouse in enumerate(probe_df_combined['mouse'].unique()):
    mdata = probe_df_combined[probe_df_combined['mouse'] == mouse]
    if mdata['group'].to_numpy()[0] == 'Sham':
        mdata.loc[:, 'day'] = 5
    fig.add_trace(go.Scattergl(x=mdata['day'] + shifts[idx], y=mdata['probe_acc'], mode='markers', marker_color='rgba(255, 255, 255, 0)',
                               opacity=0.6, showlegend=False, marker=dict(line=dict(width=1, color='black'))))
fig.update_xaxes(tickmode='array', tickvals=[5, 6], ticktext=['Sham', 'HPC'])
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.show()
fig.write_image(pjoin(fig_path, 'probe_acc_barplot.png'), width=500, height=500)

In [ ]:
## Perform a linear regression to test for differences between groups. 
data = probe_df_combined.copy()
data = probe_df_combined[probe_df_combined['group'] != 'mPFC']
model = ols('probe_acc ~ C(group)', data=data).fit()

In [ ]:
anova_table = sm.stats.anova_lm(model, typ=3)
print(anova_table)

In [ ]:
## One sided t-test
from scipy.stats import ttest_ind
x = probe_df_combined['probe_acc'][probe_df_combined['group'] == 'HPC'].to_numpy()[~pd.isna(probe_df_combined['probe_acc'][probe_df_combined['group'] == 'HPC'])]
y = probe_df_combined['probe_acc'][probe_df_combined['group'] == 'Sham'].to_numpy()
t_stat, p_val = ttest_ind(x, y, alternative='less')
print((t_stat, p_val))

### Plot probe accuracy for each group as a histogram.

In [ ]:
fig = pf.custom_graph_template(x_title='Lick Accuracy (%)', y_title='Count')
for group in ['Sham', 'mPFC', 'HPC']:
    gdata = probe_df_combined[probe_df_combined['group'] == group].reset_index(drop=True)
    fig.add_trace(go.Histogram(x=gdata['probe_acc'], name=group, marker_color=group_colors_dict[group], 
                               opacity=0.6, autobinx=False, xbins=dict(start=0, size=10),
                               marker_line_width=2, marker_line_color='black'))
fig.add_vline(x=25, line_dash='dash', line_width=1, opacity=1, line_color='black')
fig.update_yaxes(dtick=1)
fig.update_layout(barmode='overlay')
fig.show()
fig.write_image(pjoin(fig_path, 'probe_acc_histogram_3group.png'))

In [ ]:
from scipy.stats import ks_2samp 
hpc = probe_df_combined['probe_acc'][probe_df_combined['group'] == 'HPC'].to_numpy()[~pd.isna(probe_df_combined['probe_acc'][probe_df_combined['group'] == 'HPC'])]
ks_stat, p_val = ks_2samp(hpc, probe_df_combined['probe_acc'][probe_df_combined['group'] == 'Sham'])
p_val

### Plot session accuracy on day 5 vs probe accuracy on recall day

In [ ]:
fig = pf.custom_graph_template(x_title='Lick Accuracy (%)', y_title='Probe Accuracy (%)')
for group in ['Sham', 'HPC', 'mPFC']:
    sess_acc = ct_df[(ct_df['group'] == group) & (ct_df['day'] == 5)].reset_index(drop=True)
    gdata = probe_df_combined[probe_df_combined['group'] == group].reset_index(drop=True)
    fig.add_trace(go.Scattergl(x=sess_acc['lick_accuracy'], y=gdata['probe_acc'], mode='markers', 
                               marker_color=group_colors_dict[group], name=group))
fig.update_xaxes(range=[-1, 101])
fig.update_yaxes(range=[-1, 101])
fig.add_hline(y=25, line_dash='dash', line_color=chance_color, line_width=1, opacity=1)
fig.add_vline(x=25, line_dash='dash', line_color=chance_color, line_width=1, opacity=1)
fig.show()

### Look at accuracy across trials

In [ ]:
bin_size = 4
lick_thresh = 5
trial_res = {'mouse': [], 'sex': [], 'group': [], 'day': [], 'session': [], 'session_two': [], 'trial': [], 'lick_acc': []}
for exp in ['MultiCon_Lesion3', 'MultiCon_Lesion4']:
    dpath = pjoin(exp_path, f'{exp}/output/behav')
    maze_info = pd.read_csv(f'../../../{parent_dir}/{exp}/maze_yml/maze_info.csv')
    for mouse in os.listdir(dpath):
        if mouse in excluded_mice:
            pass 
        else:
            mpath = pjoin(dpath, mouse)
            sex = maze_info['Sex'][maze_info['Mouse'] == mouse].values[0]
            group = maze_info['Group'][maze_info['Mouse'] == mouse].values[0]
            if 'Sham' in group:
                group = 'Sham'
            for idx, session in enumerate(os.listdir(mpath)):
                behav = pd.read_feather(pjoin(mpath, f'{session}'))
                reward_one, reward_two = behav['reward_one'].unique()[0], behav['reward_two'].unique()[0]
                trial_acc = ctb.lick_accuracy(behav, port_list=[reward_one, reward_two], lick_threshold=lick_thresh, by_trials=True)
                
                if bin_size > 0:
                    binned_acc = ctb.bin_data(trial_acc, bin_size)
                else:
                    binned_acc = trial_acc

                for trial, val in enumerate(binned_acc):
                    trial_res['mouse'].append(mouse)
                    trial_res['sex'].append(sex)
                    trial_res['group'].append(group)
                    trial_res['day'].append(idx+1)
                    trial_res['session'].append(behav['session'].unique()[0])
                    trial_res['session_two'].append(behav['session_two'].unique()[0])
                    trial_res['trial'].append(trial+1)
                    trial_res['lick_acc'].append(val)
trial_df = pd.DataFrame(trial_res)

In [ ]:
## Plot accuracy across trials for just recall
trial_df = trial_df.replace({'AR1': 'Recall'})
day_list = ['Recall']
only_min_trials = False
ar = trial_df.copy()
avg_ar = ar.groupby(['group', 'session_two', 'trial'], as_index=False).agg({'lick_acc': ['mean', 'sem']})

fig = pf.custom_graph_template(x_title='Trials', y_title='', rows=1, columns=1, shared_x=True, width=500, height=500,
                               shared_y=True, titles=day_list)
for group in ['Sham', 'HPC', 'mPFC']:
    gdata = avg_ar[avg_ar['group'] == group].reset_index(drop=True)  
    for idx, sess in enumerate(day_list):
        trial_bins = []
        if only_min_trials:
            int_data = ar[(ar['group'] == group) & (ar['session_two'] == sess)]
            for mouse in int_data['mouse'].unique():
                d = int_data[int_data['mouse'] == mouse].reset_index(drop=True)
                trial_bins.append(d['trial'].shape[0])
                min_trials = np.min(trial_bins)
        else:
            min_trials = None

        if min_trials is not None:
            sess_data = gdata[gdata['session_two'] == sess].reset_index(drop=True)[:min_trials]
            xaxis = np.arange(1, sess_data['trial'].to_numpy()[-1] * bin_size, bin_size)[:min_trials]
            upper = sess_data['lick_acc']['mean'] + sess_data['lick_acc']['sem']
            lower = sess_data['lick_acc']['mean'] - sess_data['lick_acc']['sem']
        else:
            sess_data = gdata[gdata['session_two'] == sess].reset_index(drop=True)
            xaxis = np.arange(1, sess_data['trial'].to_numpy()[-1] * bin_size, bin_size)
            upper = sess_data['lick_acc']['mean'] + sess_data['lick_acc']['sem']
            lower = sess_data['lick_acc']['mean'] - sess_data['lick_acc']['sem']
        fig.add_trace(go.Scatter(x=xaxis, y=sess_data['lick_acc']['mean'], mode='lines', line_color=group_colors_dict[group],
                                   name=group, showlegend=False, legendgroup=group), row=1, col=idx+1)
        fig.add_trace(go.Scatter(x=xaxis, y=upper, mode='lines', marker=dict(color=error_dict[group]), legendgroup=group,
                                        name='Upper Bound', line=dict(width=0), showlegend=False), row=1, col=idx+1)
        fig.add_trace(go.Scatter(x=xaxis, y=lower, mode='lines', marker=dict(color=error_dict[group]), legendgroup=group,
                                name='Lower Bound', line=dict(width=0), showlegend=False, fillcolor=error_dict[group], fill='tonexty'), row=1, col=idx+1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color='darkgrey', opacity=1)
fig.update_yaxes(range=[0, 100])
fig.update_yaxes(title='Lick Accuracy (%)', col=1)
for val in [0, 3, 6]:
    fig.data[val]['showlegend'] = True
fig.show()

In [ ]:
## Testing new graph, didn't look how I imagined it in my head
mouse = 'mcls77'
fig = pf.custom_graph_template(x_title='', y_title='', width=700)
for session in [f'{x}' for x in np.arange(1, 6)]:
    behav = pd.read_feather(pjoin(circle_path, f'{mouse}/{mouse}_{session}.feat'))
    licks = behav[behav['lick_port'] != -1].reset_index(drop=True)
    num_licks = licks.groupby(['lick_port'], as_index=False).agg({'lick_port': 'count'})
    num_licks['port'] = [f'{x}' for x in np.arange(1, 9)]
    num_licks['lick_port'] = (num_licks['lick_port'] / num_licks['lick_port'].sum()) * 100
    fig.add_trace(go.Scatterpolar(r=num_licks['lick_port'], theta=num_licks['port'], mode='lines', 
                                  name=f'Session {session}', line_width=3))
fig.update_layout(polar=dict(radialaxis=dict(visible=True, showgrid=True, range=[0, 55]), angularaxis=dict(visible=True, showgrid=True)))
fig.show()